In [1]:
import torch
import torch.nn as nn
import nltk.tokenize
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Pre-processing

In [3]:
sh_data = open("poem_data/shakespeare.txt", "r").read()

sh_data = sh_data.lower()
sh_data = sh_data.replace("\n  ", "\n")
sh_data = sh_data.replace("\n\n\n", "\n")

for num in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]:
    sh_data = sh_data.replace(num, "")

temp1 = sh_data.split("                   ")
sh_data = temp1[1]
temp2 = sh_data.split("                 ")
sh_poems = [curr[1:] for curr in temp2]
sh_data = "".join(sh_poems)
sh_data

"from fairest creatures we desire increase,\nthat thereby beauty's rose might never die,\nbut as the riper should by time decease,\nhis tender heir might bear his memory:\nbut thou contracted to thine own bright eyes,\nfeed'st thy light's flame with self-substantial fuel,\nmaking a famine where abundance lies,\nthy self thy foe, to thy sweet self too cruel:\nthou that art now the world's fresh ornament,\nand only herald to the gaudy spring,\nwithin thine own bud buriest thy content,\nand tender churl mak'st waste in niggarding:\npity the world, or else this glutton be,\nto eat the world's due, by the grave and thee.\nwhen forty winters shall besiege thy brow,\nand dig deep trenches in thy beauty's field,\nthy youth's proud livery so gazed on now,\nwill be a tattered weed of small worth held:\nthen being asked, where all thy beauty lies,\nwhere all the treasure of thy lusty days;\nto say within thine own deep sunken eyes,\nwere an all-eating shame, and thriftless praise.\nhow much more 

In [4]:
sp_data = open("poem_data/spenser.txt", "r").read()

sp_data = sp_data.lower()
sp_data = sp_data.replace("    ", "")
sp_data = sp_data.split("\n\n")
sp_poems = [curr + "\n" for curr in sp_data if len(curr) > 25]
sp_data = "".join(sp_poems)
sp_data

"happy ye leaves when as those lily hands,\nwhich hold my life in their dead-doing might, \nshall handle you and hold in love's soft bands, \nlike captives trembling at the victor's sight. \nand happy lines, on which with starry light,\nthose lamping eyes will deign sometimes to look \nand read the sorrows of my dying sprite, \nwritten with tears in heart's close-bleeding book. \nand happy rhymes bath'd in the sacred brook,\nof helicon whence she derived is, \nwhen ye behold that angel's blessed look, \nmy soul's long-lacked food, my heaven's bliss. \nleaves, lines, and rhymes, seek her to please alone,\nwhom if ye please, i care for other none. \nunquiet thought, whom at the first i bred,\nof th' inward bale of my love-pined heart: \nand sithens have with sighs and sorrows fed, \ntill greater then my womb thou woxen art. \nbreak forth at length out of the inner part,\nin which thou lurkest like to viper's brood: \nand seek some succour both to ease my smart \nand also to sustain thy s

In [5]:
syllables_data = open("poem_data/Syllable_dictionary.txt").read()
syllables_data = syllables_data.split("\n")

syllables_dict = {}

for line in syllables_data:
    temp = line.split(" ")

    if len(temp) == 2:
        syllables_dict[temp[0]] = int(temp[1])
    elif len(temp) == 3:
        if "E" in temp[1]:
            syllables_dict[temp[0]] = int(temp[2])
        else:
            syllables_dict[temp[0]] = int(temp[1])

In [6]:
sh_poems[0]

"from fairest creatures we desire increase,\nthat thereby beauty's rose might never die,\nbut as the riper should by time decease,\nhis tender heir might bear his memory:\nbut thou contracted to thine own bright eyes,\nfeed'st thy light's flame with self-substantial fuel,\nmaking a famine where abundance lies,\nthy self thy foe, to thy sweet self too cruel:\nthou that art now the world's fresh ornament,\nand only herald to the gaudy spring,\nwithin thine own bud buriest thy content,\nand tender churl mak'st waste in niggarding:\npity the world, or else this glutton be,\nto eat the world's due, by the grave and thee.\n"

In [7]:
sh_poems[0].split(" ")

['from',
 'fairest',
 'creatures',
 'we',
 'desire',
 'increase,\nthat',
 'thereby',
 "beauty's",
 'rose',
 'might',
 'never',
 'die,\nbut',
 'as',
 'the',
 'riper',
 'should',
 'by',
 'time',
 'decease,\nhis',
 'tender',
 'heir',
 'might',
 'bear',
 'his',
 'memory:\nbut',
 'thou',
 'contracted',
 'to',
 'thine',
 'own',
 'bright',
 "eyes,\nfeed'st",
 'thy',
 "light's",
 'flame',
 'with',
 'self-substantial',
 'fuel,\nmaking',
 'a',
 'famine',
 'where',
 'abundance',
 'lies,\nthy',
 'self',
 'thy',
 'foe,',
 'to',
 'thy',
 'sweet',
 'self',
 'too',
 'cruel:\nthou',
 'that',
 'art',
 'now',
 'the',
 "world's",
 'fresh',
 'ornament,\nand',
 'only',
 'herald',
 'to',
 'the',
 'gaudy',
 'spring,\nwithin',
 'thine',
 'own',
 'bud',
 'buriest',
 'thy',
 'content,\nand',
 'tender',
 'churl',
 "mak'st",
 'waste',
 'in',
 'niggarding:\npity',
 'the',
 'world,',
 'or',
 'else',
 'this',
 'glutton',
 'be,\nto',
 'eat',
 'the',
 "world's",
 'due,',
 'by',
 'the',
 'grave',
 'and',
 'thee.\n']

In [8]:
sh_poems[0].split("\n")

['from fairest creatures we desire increase,',
 "that thereby beauty's rose might never die,",
 'but as the riper should by time decease,',
 'his tender heir might bear his memory:',
 'but thou contracted to thine own bright eyes,',
 "feed'st thy light's flame with self-substantial fuel,",
 'making a famine where abundance lies,',
 'thy self thy foe, to thy sweet self too cruel:',
 "thou that art now the world's fresh ornament,",
 'and only herald to the gaudy spring,',
 'within thine own bud buriest thy content,',
 "and tender churl mak'st waste in niggarding:",
 'pity the world, or else this glutton be,',
 "to eat the world's due, by the grave and thee.",
 '']

In [9]:
nltk.tokenize.wordpunct_tokenize(sh_poems[0])

['from',
 'fairest',
 'creatures',
 'we',
 'desire',
 'increase',
 ',',
 'that',
 'thereby',
 'beauty',
 "'",
 's',
 'rose',
 'might',
 'never',
 'die',
 ',',
 'but',
 'as',
 'the',
 'riper',
 'should',
 'by',
 'time',
 'decease',
 ',',
 'his',
 'tender',
 'heir',
 'might',
 'bear',
 'his',
 'memory',
 ':',
 'but',
 'thou',
 'contracted',
 'to',
 'thine',
 'own',
 'bright',
 'eyes',
 ',',
 'feed',
 "'",
 'st',
 'thy',
 'light',
 "'",
 's',
 'flame',
 'with',
 'self',
 '-',
 'substantial',
 'fuel',
 ',',
 'making',
 'a',
 'famine',
 'where',
 'abundance',
 'lies',
 ',',
 'thy',
 'self',
 'thy',
 'foe',
 ',',
 'to',
 'thy',
 'sweet',
 'self',
 'too',
 'cruel',
 ':',
 'thou',
 'that',
 'art',
 'now',
 'the',
 'world',
 "'",
 's',
 'fresh',
 'ornament',
 ',',
 'and',
 'only',
 'herald',
 'to',
 'the',
 'gaudy',
 'spring',
 ',',
 'within',
 'thine',
 'own',
 'bud',
 'buriest',
 'thy',
 'content',
 ',',
 'and',
 'tender',
 'churl',
 'mak',
 "'",
 'st',
 'waste',
 'in',
 'niggarding',
 ':',
 

In [10]:
sh_poems_words_syllables = [[] for i in sh_poems]

for i, poem in enumerate(sh_poems):
    lines = poem.split("\n")

    for line in lines:
        if len(line) != 0:
            words = line.split(" ")
        
            for word in words:
                if len(word) != 0:
                    word = word.replace(".", "")
                    word = word.replace(",", "")
                    word = word.replace(":", "")
                    word = word.replace(";", "")
                    word = word.replace("?", "")
                    word = word.replace("!", "")
                    word = word.replace("(", "")
                    word = word.replace(")", "")

                    if "'" in word and word not in syllables_dict:
                        word = word.replace("'", "")

                    sh_poems_words_syllables[i].append((word, syllables_dict[word]))

In [11]:
sp_poems_words_syllables = [[] for i in sp_poems]

for i, poem in enumerate(sp_poems):
    lines = poem.split("\n")

    for line in lines:
        if len(line) != 0:
            words = line.split(" ")
        
            for word in words:
                if len(word) != 0:
                    word = word.replace(".", "")
                    word = word.replace(",", "")
                    word = word.replace(":", "")
                    word = word.replace(";", "")
                    word = word.replace("?", "")
                    word = word.replace("!", "")
                    word = word.replace("(", "")
                    word = word.replace(")", "")

                    if "'" in word and word not in syllables_dict:
                        word = word.replace("'", "")

                    if word not in syllables_dict:
                        syllables_dict[word] = len(nltk.tokenize.SyllableTokenizer().tokenize(word))

                    sp_poems_words_syllables[i].append((word, syllables_dict[word]))

In [12]:
sh_poems_words_syllables[0]

[('from', 1),
 ('fairest', 2),
 ('creatures', 2),
 ('we', 1),
 ('desire', 2),
 ('increase', 2),
 ('that', 1),
 ('thereby', 2),
 ("beauty's", 2),
 ('rose', 1),
 ('might', 1),
 ('never', 2),
 ('die', 1),
 ('but', 1),
 ('as', 1),
 ('the', 1),
 ('riper', 2),
 ('should', 1),
 ('by', 1),
 ('time', 1),
 ('decease', 2),
 ('his', 1),
 ('tender', 2),
 ('heir', 1),
 ('might', 1),
 ('bear', 1),
 ('his', 1),
 ('memory', 3),
 ('but', 1),
 ('thou', 1),
 ('contracted', 3),
 ('to', 1),
 ('thine', 1),
 ('own', 1),
 ('bright', 1),
 ('eyes', 1),
 ("feed'st", 1),
 ('thy', 1),
 ("light's", 1),
 ('flame', 1),
 ('with', 1),
 ('self-substantial', 4),
 ('fuel', 1),
 ('making', 2),
 ('a', 1),
 ('famine', 2),
 ('where', 1),
 ('abundance', 3),
 ('lies', 1),
 ('thy', 1),
 ('self', 1),
 ('thy', 1),
 ('foe', 1),
 ('to', 1),
 ('thy', 1),
 ('sweet', 1),
 ('self', 1),
 ('too', 1),
 ('cruel', 2),
 ('thou', 1),
 ('that', 1),
 ('art', 1),
 ('now', 1),
 ('the', 1),
 ("world's", 1),
 ('fresh', 1),
 ('ornament', 3),
 ('and', 

In [13]:
sp_poems_words_syllables[0]

[('happy', 2),
 ('ye', 1),
 ('leaves', 1),
 ('when', 1),
 ('as', 1),
 ('those', 1),
 ('lily', 2),
 ('hands', 1),
 ('which', 1),
 ('hold', 1),
 ('my', 1),
 ('life', 1),
 ('in', 1),
 ('their', 1),
 ('dead-doing', 3),
 ('might', 1),
 ('shall', 1),
 ('handle', 2),
 ('you', 1),
 ('and', 1),
 ('hold', 1),
 ('in', 1),
 ("love's", 1),
 ('soft', 1),
 ('bands', 1),
 ('like', 1),
 ('captives', 3),
 ('trembling', 2),
 ('at', 1),
 ('the', 1),
 ('victors', 2),
 ('sight', 1),
 ('and', 1),
 ('happy', 2),
 ('lines', 1),
 ('on', 1),
 ('which', 1),
 ('with', 1),
 ('starry', 2),
 ('light', 1),
 ('those', 1),
 ('lamping', 2),
 ('eyes', 1),
 ('will', 1),
 ('deign', 1),
 ('sometimes', 2),
 ('to', 1),
 ('look', 1),
 ('and', 1),
 ('read', 1),
 ('the', 1),
 ('sorrows', 2),
 ('of', 1),
 ('my', 1),
 ('dying', 2),
 ('sprite', 2),
 ('written', 2),
 ('with', 1),
 ('tears', 1),
 ('in', 1),
 ("heart's", 1),
 ('close-bleeding', 5),
 ('book', 1),
 ('and', 1),
 ('happy', 2),
 ('rhymes', 2),
 ('bathd', 1),
 ('in', 1),
 ('

# Model Training

In [14]:
sh_chars = sorted(list(set(sh_data)))
data_size, vocab_size = len(sh_data), len(sh_chars)
print("----------------------------------------")
print("Data has {} characters, {} unique".format(data_size, vocab_size))
print("----------------------------------------")

# char to index and index to char maps
sh_char_to_ix = { ch:i for i,ch in enumerate(sh_chars) }
sh_ix_to_char = { i:ch for i,ch in enumerate(sh_chars) }

----------------------------------------
Data has 93673 characters, 38 unique
----------------------------------------


In [15]:
sh_poems_ix = [0 for i in range(len(sh_poems))]

for i, poem in enumerate(sh_poems):
    poem = list(poem)
    for j, ch in enumerate(poem):
        poem[j] = sh_char_to_ix[ch]
        
    poem = torch.tensor(poem)
    sh_poems_ix[i] = poem

In [16]:
sp_chars = sorted(list(set(sp_data)))
data_size, vocab_size = len(sp_data), len(sp_chars)
print("----------------------------------------")
print("Data has {} characters, {} unique".format(data_size, vocab_size))
print("----------------------------------------")

# char to index and index to char maps
sp_char_to_ix = { ch:i for i,ch in enumerate(sp_chars) }
sp_ix_to_char = { i:ch for i,ch in enumerate(sp_chars) }

----------------------------------------
Data has 59538 characters, 38 unique
----------------------------------------


In [17]:
sp_poems_ix = [0 for i in range(len(sp_poems))]

for i, poem in enumerate(sp_poems):
    poem = list(poem)
    for j, ch in enumerate(poem):
        poem[j] = sp_char_to_ix[ch]
        
    poem = torch.tensor(poem)
    sp_poems_ix[i] = poem

In [18]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, output_size, hidden_size):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size)
        self.decoder = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.Softmax()
    
    def forward(self, input_seq, hidden_state):
        embedding = self.embedding(input_seq)
        output, hidden_state = self.rnn(embedding, hidden_state)
        output = self.decoder(output)
#         output = self.softmax(output)
        return output, (hidden_state[0].detach(), hidden_state[1].detach())

In [19]:
model = RNN(input_size = vocab_size, embedding_size = vocab_size, output_size = vocab_size, hidden_size = 150)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1)

In [20]:
for i_epoch in range(1, 11):
    n = 0
    running_loss = 0
    
    for poem in sh_poems_ix:
        for i in range(40, len(poem) - 1, 10):
            hidden_state = None
            input_seq = poem[i - 40 : i]
            target_seq = poem[i - 40 + 1 : i + 1]
            
            output, _ = model(input_seq, hidden_state)
            
            loss = loss_fn(torch.squeeze(output), torch.squeeze(target_seq))
            running_loss += loss.item()
            n += 1
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    print("Epoch: {0} \t Loss: {1:.8f}".format(i_epoch, running_loss/n))

Epoch: 1 	 Loss: 70.02042529
Epoch: 2 	 Loss: 69.25438886
Epoch: 3 	 Loss: 68.93366781
Epoch: 4 	 Loss: 66.64717762
Epoch: 5 	 Loss: 67.82140791
Epoch: 6 	 Loss: 68.74099454
Epoch: 7 	 Loss: 69.38351749
Epoch: 8 	 Loss: 68.05812615
Epoch: 9 	 Loss: 67.64049609
Epoch: 10 	 Loss: 67.83499663


In [21]:
model = RNN(input_size = vocab_size, embedding_size = vocab_size, output_size = vocab_size, hidden_size = 150)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)

In [22]:
for i_epoch in range(1, 11):
    n = 0
    running_loss = 0
    
    for poem in sh_poems_ix:
        for i in range(40, len(poem) - 1, 10):
            hidden_state = None
            input_seq = poem[i - 40 : i]
            target_seq = poem[i - 40 + 1 : i + 1]
            
            output, _ = model(input_seq, hidden_state)
            
            loss = loss_fn(torch.squeeze(output), torch.squeeze(target_seq))
            running_loss += loss.item()
            n += 1
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    print("Epoch: {0} \t Loss: {1:.8f}".format(i_epoch, running_loss/n))

Epoch: 1 	 Loss: 4.27880101
Epoch: 2 	 Loss: 4.26535983
Epoch: 3 	 Loss: 4.47952566
Epoch: 4 	 Loss: 4.42351814
Epoch: 5 	 Loss: 4.30575475
Epoch: 6 	 Loss: 4.12550972
Epoch: 7 	 Loss: 4.26157586
Epoch: 8 	 Loss: 4.23873205
Epoch: 9 	 Loss: 4.22602354
Epoch: 10 	 Loss: 4.21532944


In [23]:
model = RNN(input_size = vocab_size, embedding_size = vocab_size, output_size = vocab_size, hidden_size = 150)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [24]:
for i_epoch in range(1, 11):
    n = 0
    running_loss = 0
    
    for poem in sh_poems_ix:
        for i in range(40, len(poem) - 1, 10):
            hidden_state = None
            input_seq = poem[i - 40 : i]
            target_seq = poem[i - 40 + 1 : i + 1]
            
            output, _ = model(input_seq, hidden_state)
            
            loss = loss_fn(torch.squeeze(output), torch.squeeze(target_seq))
            running_loss += loss.item()
            n += 1
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    print("Epoch: {0} \t Loss: {1:.8f}".format(i_epoch, running_loss/n))

Epoch: 1 	 Loss: 2.07098191
Epoch: 2 	 Loss: 2.02504313
Epoch: 3 	 Loss: 2.03218175
Epoch: 4 	 Loss: 2.05471860
Epoch: 5 	 Loss: 2.06071207
Epoch: 6 	 Loss: 2.06369827
Epoch: 7 	 Loss: 2.06390966
Epoch: 8 	 Loss: 2.06392601
Epoch: 9 	 Loss: 2.08410020
Epoch: 10 	 Loss: 2.09885689


In [25]:
model = RNN(input_size = vocab_size, embedding_size = vocab_size, output_size = vocab_size, hidden_size = 150)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [26]:
for i_epoch in range(1, 11):
    n = 0
    running_loss = 0
    
    for poem in sh_poems_ix:
        for i in range(40, len(poem) - 1, 10):
            hidden_state = None
            input_seq = poem[i - 40 : i]
            target_seq = poem[i - 40 + 1 : i + 1]
            
            output, _ = model(input_seq, hidden_state)
            
            loss = loss_fn(torch.squeeze(output), torch.squeeze(target_seq))
            running_loss += loss.item()
            n += 1
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    print("Epoch: {0} \t Loss: {1:.8f}".format(i_epoch, running_loss/n))

Epoch: 1 	 Loss: 1.87099182
Epoch: 2 	 Loss: 1.63022152
Epoch: 3 	 Loss: 1.55109995
Epoch: 4 	 Loss: 1.49892978
Epoch: 5 	 Loss: 1.46555991
Epoch: 6 	 Loss: 1.43809730
Epoch: 7 	 Loss: 1.41261293
Epoch: 8 	 Loss: 1.39487872
Epoch: 9 	 Loss: 1.37656161
Epoch: 10 	 Loss: 1.36250807


In [27]:
model = RNN(input_size = vocab_size, embedding_size = vocab_size, output_size = vocab_size, hidden_size = 150)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [28]:
for i_epoch in range(1, 11):
    n = 0
    running_loss = 0
    
    for poem in sh_poems_ix:
        for i in range(40, len(poem) - 1, 10):
            hidden_state = None
            input_seq = poem[i - 40 : i]
            target_seq = poem[i - 40 + 1 : i + 1]
            
            output, _ = model(input_seq, hidden_state)
            
            loss = loss_fn(torch.squeeze(output), torch.squeeze(target_seq))
            running_loss += loss.item()
            n += 1
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    print("Epoch: {0} \t Loss: {1:.8f}".format(i_epoch, running_loss/n))

Epoch: 1 	 Loss: 2.32877847
Epoch: 2 	 Loss: 1.98595418
Epoch: 3 	 Loss: 1.86457810
Epoch: 4 	 Loss: 1.78901454
Epoch: 5 	 Loss: 1.73350475
Epoch: 6 	 Loss: 1.68585677
Epoch: 7 	 Loss: 1.64700036
Epoch: 8 	 Loss: 1.61463651
Epoch: 9 	 Loss: 1.58662214
Epoch: 10 	 Loss: 1.56177236


In [29]:
model = RNN(input_size = vocab_size, embedding_size = vocab_size, output_size = vocab_size, hidden_size = 100)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [30]:
for i_epoch in range(1, 11):
    n = 0
    running_loss = 0
    
    for poem in sh_poems_ix:
        for i in range(40, len(poem) - 1, 10):
            hidden_state = None
            input_seq = poem[i - 40 : i]
            target_seq = poem[i - 40 + 1 : i + 1]
            
            output, _ = model(input_seq, hidden_state)
            
            loss = loss_fn(torch.squeeze(output), torch.squeeze(target_seq))
            running_loss += loss.item()
            n += 1
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    print("Epoch: {0} \t Loss: {1:.8f}".format(i_epoch, running_loss/n))

Epoch: 1 	 Loss: 1.93437265
Epoch: 2 	 Loss: 1.69388813
Epoch: 3 	 Loss: 1.61332938
Epoch: 4 	 Loss: 1.56798859
Epoch: 5 	 Loss: 1.53042937
Epoch: 6 	 Loss: 1.50367993
Epoch: 7 	 Loss: 1.48300990
Epoch: 8 	 Loss: 1.46499563
Epoch: 9 	 Loss: 1.45086706
Epoch: 10 	 Loss: 1.43890557


In [31]:
model = RNN(input_size = vocab_size, embedding_size = vocab_size, output_size = vocab_size, hidden_size = 200)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [33]:
for i_epoch in range(1, 11):
    n = 0
    running_loss = 0
    
    for poem in sh_poems_ix:
        for i in range(40, len(poem) - 1, 10):
            hidden_state = None
            input_seq = poem[i - 40 : i]
            target_seq = poem[i - 40 + 1 : i + 1]
            
            output, _ = model(input_seq, hidden_state)
            
            loss = loss_fn(torch.squeeze(output), torch.squeeze(target_seq))
            running_loss += loss.item()
            n += 1
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    print("Epoch: {0} \t Loss: {1:.8f}".format(i_epoch, running_loss/n))

Epoch: 1 	 Loss: 1.62063466
Epoch: 2 	 Loss: 1.52541300
Epoch: 3 	 Loss: 1.46460711
Epoch: 4 	 Loss: 1.41875935
Epoch: 5 	 Loss: 1.38237491
Epoch: 6 	 Loss: 1.35665760
Epoch: 7 	 Loss: 1.33074247
Epoch: 8 	 Loss: 1.30891792
Epoch: 9 	 Loss: 1.28867280
Epoch: 10 	 Loss: 1.27411264


In [34]:
model = RNN(input_size = vocab_size, embedding_size = vocab_size, output_size = vocab_size, hidden_size = 200)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [35]:
for i_epoch in range(1, 51):
    n = 0
    running_loss = 0
    
    for poem in sh_poems_ix:
        for i in range(40, len(poem) - 1, 10):
            hidden_state = None
            input_seq = poem[i - 40 : i]
            target_seq = poem[i - 40 + 1 : i + 1]
            
            output, _ = model(input_seq, hidden_state)
            
            loss = loss_fn(torch.squeeze(output), torch.squeeze(target_seq))
            running_loss += loss.item()
            n += 1
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    print("Epoch: {0} \t Loss: {1:.8f}".format(i_epoch, running_loss/n))

Epoch: 1 	 Loss: 1.84504402
Epoch: 2 	 Loss: 1.60880724
Epoch: 3 	 Loss: 1.51903234
Epoch: 4 	 Loss: 1.46436188
Epoch: 5 	 Loss: 1.42072945
Epoch: 6 	 Loss: 1.38860780
Epoch: 7 	 Loss: 1.36022966
Epoch: 8 	 Loss: 1.33578576
Epoch: 9 	 Loss: 1.31564089
Epoch: 10 	 Loss: 1.29829126
Epoch: 11 	 Loss: 1.28118184
Epoch: 12 	 Loss: 1.27085117
Epoch: 13 	 Loss: 1.25243666
Epoch: 14 	 Loss: 1.24438171
Epoch: 15 	 Loss: 1.22989492
Epoch: 16 	 Loss: 1.22064034
Epoch: 17 	 Loss: 1.21526489
Epoch: 18 	 Loss: 1.20635388
Epoch: 19 	 Loss: 1.19757575
Epoch: 20 	 Loss: 1.19703239
Epoch: 21 	 Loss: 1.19010373
Epoch: 22 	 Loss: 1.18530670
Epoch: 23 	 Loss: 1.18238086
Epoch: 24 	 Loss: 1.18105458
Epoch: 25 	 Loss: 1.17847618
Epoch: 26 	 Loss: 1.17575817
Epoch: 27 	 Loss: 1.17412900
Epoch: 28 	 Loss: 1.17315379
Epoch: 29 	 Loss: 1.17098351
Epoch: 30 	 Loss: 1.16747401
Epoch: 31 	 Loss: 1.16835813
Epoch: 32 	 Loss: 1.16820039
Epoch: 33 	 Loss: 1.17117093
Epoch: 34 	 Loss: 1.16604038
Epoch: 35 	 Loss: 1.161

In [37]:
prompt = "shall i compare thee to a summer's day?\n"

prompt = list(prompt)
for i, ch in enumerate(prompt):
    prompt[i] = sh_char_to_ix[ch]

with torch.no_grad():
#     prompt = torch.tensor(prompt).to(device)
    prompt = torch.tensor(prompt)
    hidden_init = None
    output, hidden = model(prompt, hidden_init)

    for _ in range(1_000):
        output = output[-1]
        prediction = torch.argmax(nn.functional.softmax(output / 0.25, dim = 0))
        print(sh_ix_to_char[int(prediction.detach().numpy())],end="")
        output, hidden = model(torch.tensor([prediction]), hidden)

thou be character sun thy sweet buds have sweet brow,
for thee i am attent,
and there is but thou betrayed the basest thee i am at a face,
for thee is brow of bath the basest of this bark,
the basest doth lie honour face shall be forgot,
which have sweet flattery in the breast thee are not thy breast,
no blunt past thee as fair thee as fair, but then so my brain and for my self away,
the basest doth lie to be can be then my heart that i am at fair thee from my fair:
a thinking thee are not the basest of this bare thee as fairest thee and there with thy sweet flate,
whilst thou that i am at frown the better than thine eyes be then my heart thou be forgot,
which i love thee my self i do count bare,
which have sweet love thee my self away, and there is brow?
why lov'st the lays of thee that i am at frown the basest be the lily i note,
but from thee are brief thee a face,
when i am at from thee are not be fairs of true mine eyes?
but then be comment have i seen,
for that i am at fair chide

# Improving recurrent models

In [38]:
sh_words = set()

for poem in sh_poems:
    for line in poem.split("\n"):
        for word in line.split(" "):
            sh_words.add(word)
        
sh_words = sorted(list(sh_words))

sh_word_to_ix = {w:i for i, w in enumerate(sh_words)}
sh_ix_to_word = {i:w for i, w in enumerate(sh_words)}

In [39]:
sh_poems_ix_words = [0 for i in range(len(sh_poems))]

for i, poem in enumerate(sh_poems):
    result = []
    for line in poem.split("\n"):
        for word in line.split(" "):
            result.append(sh_word_to_ix[word])
    
    sh_poems_ix_words[i] = torch.tensor(result)

In [40]:
model_words = RNN(input_size = len(sh_words), embedding_size = len(sh_words), output_size = len(sh_words), hidden_size = 200)
loss_fn_words = nn.CrossEntropyLoss()
optimizer_words = torch.optim.Adam(model_words.parameters(), lr = 0.001)

In [41]:
for i_epoch in range(1, 11):
    n = 0
    running_loss = 0
    
    for poem in sh_poems_ix_words:
        for i in range(10, len(poem) - 1, 10):
            hidden_state = None
            input_seq = poem[i - 10 : i]
            target_seq = poem[i - 10 + 1 : i + 1]
            
            output, _ = model_words(input_seq, hidden_state)
            
            loss = loss_fn_words(torch.squeeze(output), torch.squeeze(target_seq))
            running_loss += loss.item()
            n += 1
            
            optimizer_words.zero_grad()
            loss.backward()
            optimizer_words.step()
            
    print("Epoch: {0} \t Loss: {1:.8f}".format(i_epoch, running_loss/n))

Epoch: 1 	 Loss: 7.19251193
Epoch: 2 	 Loss: 5.28098241
Epoch: 3 	 Loss: 3.98357080
Epoch: 4 	 Loss: 3.00085790
Epoch: 5 	 Loss: 2.31318516
Epoch: 6 	 Loss: 1.85934984
Epoch: 7 	 Loss: 1.55439455
Epoch: 8 	 Loss: 1.31366310
Epoch: 9 	 Loss: 1.10491412
Epoch: 10 	 Loss: 0.94607592


In [43]:
prompt = "shall i compare thee to a summer's day?\n"
result = []

for line in prompt.split("\n"):
    for word in line.split(" "):
        result.append(sh_word_to_ix[word])

print(result)
# prompt = list(prompt)
# for i, ch in enumerate(prompt):
#     prompt[i] = sh_char_to_ix[ch]

with torch.no_grad():
#     prompt = torch.tensor(prompt).to(device)
    prompt = torch.tensor(result)
    hidden_init = None
    output, hidden = model_words(prompt, hidden_init)

    result_str = ""
    
    for _ in range(1000):
        output = output[-1]
        prediction = torch.argmax(nn.functional.softmax(output / 0.25, dim = 0)) 
        result_str += sh_ix_to_word[int(prediction.detach().numpy())] + " "
        print(sh_ix_to_word[int(prediction.detach().numpy())],end=" ")
        output, hidden = model_words(torch.tensor([prediction]), hidden)

[3197, 1861, 691, 3654, 3767, 49, 3540, 846, 0]
all the breathers of this world are dead, you still with my heart to groan for that which it contains, and that is i my self with present moan? what merit in the breath that thou mayst take, all the world to say this will one will pluck thee back, she keeps thee that to my breast, i sick withal the help of bath desired, and thither hied a sad distempered guest. but found no cure, the bath for my help lies, where is my love to-day, to-morrow kind, still constant in them more hath in my love is lost. for i have sworn deep oaths of thy love, and i desperate now approve, desire is death, which physic did except. past cure i must the very part was i should despair i should grow mad, and in my madness might speak ill of thee, my heart my true sight, and swear that brightness doth not to thy love, thy love, happy to die! but what's so blessed-fair that fears no blot? thou mayst take, all this away, and my love is as a fever longing still, for th

In [44]:
# Calculating Perplexity 
# (Code Inspired by Response Given by User 'mujjiga' on https://stackoverflow.com/questions/54941966/how-can-i-calculate-perplexity-using-nltk)


# Improved Word-Based Model

sh_words_freq = {}

for poem in sh_poems:
    for line in poem.split("\n"):
        for word in line.split(" "):
            if word in sh_words_freq:
                sh_words_freq[word] += 1
            else:
                sh_words_freq[word] = 1

perplexity_words = 0
result = "in thy heart that thou mayst take, all this away,".split(" ")

for word in result:
    perplexity_words += np.log2(sh_words_freq[word] / len(sh_words))
    
perplexity_words /= len(result)
perplexity_words = np.power(2, -perplexity_words)
print("Perplexity of Improved Word-Based Model: " + str(perplexity_words))


# Old Character-Based Model

sh_chars_freq = {}

for poem in sh_poems:
    for char in list(poem):
        if char in sh_chars_freq:
            sh_chars_freq[char] += 1
        else:
            sh_chars_freq[char] = 1
        
perplexity_chars = 0
result = "when i be thy self away, and they see th"

for char in list(result):
    perplexity_chars += np.log2(sh_chars_freq[char] / len(sh_chars))
    
perplexity_chars /= len(list(result))
perplexity_chars = np.power(2, -perplexity_chars)
print("Perplexity of Old Character-Based Model: " + str(perplexity_chars))

Perplexity of Improved Word-Based Model: 68.7167142987722
Perplexity of Old Character-Based Model: 0.006640230781835659
